##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Pipeline と TensorFlow Model Analysis を使ったモデル解析


注意: このチュートリアルを Colab ノートブックで実行することをお勧めします。セットアップは必要ありません。「Google Colab で実行」をクリックしてください。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfma"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_tfma.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_tfma.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

ノートブックを使用したこのチュートリアルでは、単純な分類モデルを作成して複数のランでパフォーマンスを分析する TFX パイプラインを作成して実行します。このノートブックは[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で構築した TFX パイプラインを基盤として使用しています。そのチュートリアルにまだ目を通していない場合は、このノートブックを進める前に読むことをお勧めします。

新しいデータセットでモデルを調整したり、トレーニングしたりする過程で、モデルが改善されたか、悪化したかをチェックする必要がありますが、精度などのトップレベルの指標をチェックするだけでは不十分な場合があります。トレーニング済みのモデルは、本番に進められる前に評価が必要があります。

前のチュートリアルで作成したパイプラインに、`Evaluator` コンポーネントを追加しましょう。Evaluator コンポーネントはモデルの詳細な分析を実行し、ベースラインモデルに対して新しいモデルを比較することで、「十分」であるかどうかを判定します。これは、[TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma) ライブラリを使って実装します。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。

## セットアップ

セットアッププロセスは、前のチュートリアルと同じです。

まず、TFX Python パッケージをインストールし、モデルに使用するデータセットをダウンロードする必要があります。

### Pip のアップグレード

ローカルで実行する場合にシステムの Pip をアップグレードしないように、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップふれーどできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX をインストールする


In [ ]:
!pip install -U tfx

### shapely をアンインストールする

TODO(b/263441833) これは、InportError を回避するための一時的な策です。最終的には、追加の依存関係をアンインストールする代わりに Bigquery の最近のバージョンをサポートすることで処理される必要があります。

In [ ]:
!pip uninstall shapely -y

### ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行した後に、「RESTART RUNTIME」ボタンをクリックするか、「Runtime」&gt;「Restart runtime ...」メニューを使って、ランタイムを再起動する必要があります。これは、Colab がパッケージを読み込むために必要な作業です。

TensorFlow と TFX のバージョンを確認します。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 変数をセットアップする

パイプラインを定義するために使用する変数がいくつかあります。これらの変数は、必要に応じてカスタマイズすることが可能です。デフォルトでは、パイプラインからのすべての出力は、現在のディレクトリに生成されます。

In [ ]:
import os

PIPELINE_NAME = "penguin-tfma"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### サンプルデータを準備する

同じ [Palmer Penguins データセット](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)を使用します。

このデータセットには、範囲 [0,1] を持つようにすでに正規化された数値特徴量が 4 つあります。ペンギンの `species`（種）を予測する分類モデルを構築します。

TFX ExampleGen はディレクトリから入力を読み取るため、ディレクトリを作成してデータセットをそれにコピーする必要があります。

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

## パイプラインを作成する

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)で作成したパイプラインに [`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) コンポーネントを追加します。

Evaluator コンポーネントには、`ExampleGen` コンポーネントの入力データと `Trainer` コンポーネントと [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) オブジェクトのモデルが必要です。オプションとして、トレーニング済みの新しいモデルと指標を比較するためのベースラインモデルを提供することができます。

Evaluator は、`ModelEvaluation` と `ModelBlessing` の 2 種類の出力アーティファクトを作成します。ModelEvaluation には、TFMA ライブラリでさらに調査と可視化を行える詳細な評価結果が含まれます。ModelBlessing には、モデルが特定の基準を満たしたかどうか、そして後で Pusher などのコンポーネントでシグナルとして使用できるかに関するブール値の結果が含まれます。


### モデルトレーニングのコードを記述する

[単純な TFX パイプラインのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)と同じモデルコードを使用します。

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

### パイプライン定義を記述する

TFX パイプラインを作成する関数を定義します。前述の Evaluator コンポーネントの他に、[`Resolver`](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/dsl/Resolver) というもう 1 つのノードを追加します。新しいモデルが前のモデルよりも改善されているかを確認するには、新しいモデルを「ベースライン」というプッシュ済みの前のモデルに比較する必要があります。[ML Metadata（MLMD）](https://www.tensorflow.org/tfx/guide/mlmd) はパイプラインの以前のアーティファクトすべてを追跡し、`Resolver` は、<code>LatestBlessedModelStrategy</code> というストラテジーを使って、MLMD の最後の <em>blessed</em> モデル（Evaluator を合格したモデル）がどれであったかを見つけることができます。


In [ ]:
import tensorflow_model_analysis as tfma

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # NEW: Get the latest blessed model for Evaluator.
  model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

  # NEW: Uses TFMA to compute evaluation statistics over features of a model and
  #   perform quality validation of a candidate model (compared to a baseline).

  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='species')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['species']),
          ],
      metrics_specs=[
          tfma.MetricsSpec(per_slice_thresholds={
              'sparse_categorical_accuracy':
                  tfma.PerSliceMetricThresholds(thresholds=[
                      tfma.PerSliceMetricThreshold(
                          slicing_specs=[tfma.SlicingSpec()],
                          threshold=tfma.MetricThreshold(
                              value_threshold=tfma.GenericValueThreshold(
                                   lower_bound={'value': 0.6}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}))
                       )]),
          })],
      )
  evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'], # Pass an evaluation result.
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,

      # Following two components were added to the pipeline.
      model_resolver,
      evaluator,

      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

Evaluator には、`eval_config` を介して、以下の情報を提供する必要があります。

- 構成が必要なその他の指標（モデルに定義済みの指標の他に希望する場合）。
- 構成が必要なスライス
- 検証を含める場合、検証する必要のあるモデルの検証しきい値

`SparseCategoricalAccuracy` はすでに `model.compile()` 呼び出しに含まれているため、自動的に分析に含められます。そのため、ここでは他の指標を追加する必要はありません。このモデルが十分であるかどうかについても、`SparseCategoricalAccuracy` で判定されます。

データセット全体とペンギンの各種の指標を計算します。`SlicingSpec` によって、宣言した指標をどのように集計するかを指定します。

新しいモデルが合格する必要のあるしきい値が 2 つあります。1 つは絶対しきい値の 0.6 で、もう 1 つはベースラインよりも高くなる必要のある相対しきい値です。初めてパイプラインを実行する際は、`change_threshold` は無視され、value_threshold のみがチェックされます。パイプラインが 2 回以上実行される場合は、`Resolver` が前のランのモデルを見つけ、そのモデルがベースラインとして比較に使用されます。

詳細は、[Evaluator コンポーネントガイド](https://www.tensorflow.org/tfx/guide/evaluator#using_the_evaluator_component)をご覧ください。

## パイプラインを実行する


前のチュートリアルのように、`LocalDagRunner` を使用します。

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

パイプラインが完了すると、以下のように表示されます。

```
INFO:absl:Blessing result True written to pipelines/penguin-tfma/Evaluator/blessing/4.
```

または手動で、生成されたアーティファクトが格納される出力ディレクトリをチェックすることもできます。ファイルブラウザーで`pipelines/penguin-tfma/Evaluator/blessing/` にアクセスすると、評価結果に応じて、`BLESSED` または `NOT_BLESSED` の名前でファイルが表示されます。

祝福の結果が `False` である場合、Pusher はモデルを `serving_model_dir` にプッシュすることを拒否します。モデルが本番での使用に十分でないためです。

できれば異なる評価構成を使用して、パイプラインをもう一度実行することができます。全く同じ構成とデータセットでパイプラインを実行しても、モデルトレーニングの継承されたランダム性により、わずかに異なるトレーニング済みモデルが得られ、`NOT_BLESSED` モデルとなる場合があります。

### パイプラインの出力を調べる

TFMA を使用して、ModelEvaluation アーティファクトを調べ、評価結果を可視化することができます。

> **注意: Colab を使用していない場合は、Jupyter Extensions** をインストールしてください。TFMA から可視化を確認するには、TensorFlow Model Analysis 拡張機能が必要です。この拡張機能は、Google Colab にインストール済みですが、このノートブックを他の環境で実行している場合には、必要となる場合があります。Jupyter 拡張機能のインストール手順は、[インストールガイド](https://github.com/tensorflow/model-analysis#installation)をご覧ください。


#### 出力アーティファクトから分析結果を取得する

MLMD API を使用して、プログラムでこれらの出力の場所を特定することができます。まず、生成されたばかりの出力アーティファクトを検索するユーティリティ関数を定義しましょう。

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)


`Evaluator` コンポーネントの最新の実行を見つけ、その出力アーティファクトを取得します。

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  evaluator_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                          'Evaluator')
  eval_artifact = evaluator_output[standard_component_specs.EVALUATION_KEY][0]

`Evaluator` は必ず 1 つの評価アーティファクトを返し、それを TensorFlow Model Analysis ライブラリで可視化することができます。たとえば、以下は、ペンギンの各種の精度指標をレンダリングするコードです。

In [ ]:
import tensorflow_model_analysis as tfma

eval_result = tfma.load_eval_result(eval_artifact.uri)
tfma.view.render_slicing_metrics(eval_result, slicing_column='species')

`Show` ドロップダウンリストで 'sparse_categorical_accuracy' を選択すると、種ごとの精度値を確認できます。より多くのスライスを追加し、モデルがすべての分布において十分であるか、潜在的なバイアスがないかどうかを確認するとよいでしょう。

## 次のステップ

モデル分析の詳細については、[TensorFlow Model Analysis ライブラリのチュートリアル](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic)をご覧ください。

その他のリソースは、https://www.tensorflow.org/tfx/tutorials に掲載されています。

TFX の様々な概念についての詳細は、[TFX パイプラインの理解](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)をご覧ください。
